# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()
import pandas as pd

In [ ]:
# Your code here; create the connection and cursor

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [5]:
thing1 = cur.execute('''
    SELECT customerNumber, contactFirstName, contactLastName
    FROM customers
    WHERE (
        SELECT orderDate
        FROM orders
        WHERE orderDate = '2003-01-31'
    )
''').fetchall()
df = pd.DataFrame(thing1)
df

,0,1,2
0,103,Carine,Schmitt
1,112,Jean,King
2,114,Peter,Ferguson
3,119,Janine,Labrune
4,121,Jonas,Bergulfsen
...,...,...,...
117,486,Rosa,Salazar
118,487,Sue,Taylor
119,489,Thomas,Smith
120,495,Valarie,Franco


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [64]:
thing1 = cur.execute('''
    SELECT products.productName, count(customerNumber) AS total
    FROM products
    JOIN orderdetails
    USING(productCode)
    JOIN orders
    USING(orderNumber)
    WHERE customerNumber IN (
        SELECT DISTINCT customerNumber
        FROM orders
    )
    GROUP BY productName
''').fetchall()
df = pd.DataFrame(thing1)
df.columns = [x[0] for x in cur.description]
df

,productName,total
0,18th Century Vintage Horse Carriage,28
1,18th century schooner,27
2,1900s Vintage Bi-Plane,28
3,1900s Vintage Tri-Plane,28
4,1903 Ford Model A,27
...,...,...
104,The Mayflower,27
105,The Queen Mary,27
106,The Schooner Bluenose,27
107,The Titanic,27


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [62]:
thing1 = cur.execute('''
    SELECT products.productName, quantityOrdered
    FROM products
    JOIN orderdetails
    USING(productCode)
        
''').fetchall()
df = pd.DataFrame(thing1)
df.columns = [x[0] for x in cur.description]
df

,productName,quantityOrdered
0,1969 Harley Davidson Ultimate Chopper,20
1,1969 Harley Davidson Ultimate Chopper,21
2,1969 Harley Davidson Ultimate Chopper,22
3,1969 Harley Davidson Ultimate Chopper,23
4,1969 Harley Davidson Ultimate Chopper,23
...,...,...
2991,Pont Yacht,42
2992,Pont Yacht,43
2993,Pont Yacht,47
2994,Pont Yacht,50


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [85]:
thing1 = cur.execute('''
    SELECT employeeNumber, firstName, lastName, offices.city, officeCode
    FROM employees
    JOIN offices
    USING(officeCode)
    JOIN customers
    ON customers.salesRepEmployeeNumber = employees.employeeNumber
    JOIN(
        SELECT orderdetails.productCode,count(orders.customerNumber) as totalCustomers
        FROM orderdetails
        JOIN orders
        USING (orderNumber)
        GROUP BY productCode
    )
    WHERE totalCustomers < 20
        
''').fetchall()
df = pd.DataFrame(thing1)
df.columns = [x[0] for x in cur.description]
df

ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [94]:
thing1 = cur.execute('''
    SELECT employeenumber, firstName, lastName, count(customerNumber) as numberCustomers
    FROM employees e
    JOIN customers c
    ON salesRepEmployeeNumber = employeeNumber
    WHERE salesRepEmployeeNumber IN (
        SELECT salesRepEmployeeNumber
        FROM customers
        group by salesRepEmployeeNumber
        HAVING creditLimit > 15000     
    )
    group by employeeNumber
    
        
''').fetchall()
df = pd.DataFrame(thing1)
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,numberCustomers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!